In [61]:
import jinja2
import json
import ipywidgets as ipw

In [62]:
import subprocess

In [63]:
import IPython

In [64]:
from notebook import notebookapp
servers = list(notebookapp.list_running_servers())

In [65]:
import ipykernel

In [66]:
import jupyter_client

In [67]:
import numpy as np

In [68]:
A = np.arange(12).reshape(6, 2)

In [72]:
A

array([[ 0,  1],
       [ 2,  3],
       [ 4,  5],
       [ 6,  7],
       [ 8,  9],
       [10, 11]])

In [81]:
n, m = A.shape

In [82]:
diff = np.zeros([n, n])

In [92]:
np.broadcast?

In [89]:
diff.reshape(n**2)

array([  0.        ,   2.82842712,   5.65685425,   8.48528137,
        11.3137085 ,  14.14213562,   2.82842712,   0.        ,
         2.82842712,   5.65685425,   8.48528137,  11.3137085 ,
         5.65685425,   2.82842712,   0.        ,   2.82842712,
         5.65685425,   8.48528137,   8.48528137,   5.65685425,
         2.82842712,   0.        ,   2.82842712,   5.65685425,
        11.3137085 ,   8.48528137,   5.65685425,   2.82842712,
         0.        ,   2.82842712,  14.14213562,  11.3137085 ,
         8.48528137,   5.65685425,   2.82842712,   0.        ])

In [69]:
np.subtract.outer(A, A, axis=1)

TypeError: 'axis' is an invalid keyword to ufunc 'subtract'

In [55]:
list(np.broadcast(A, A))

[(0, 0),
 (1, 1),
 (2, 2),
 (3, 3),
 (4, 4),
 (5, 5),
 (6, 6),
 (7, 7),
 (8, 8),
 (9, 9),
 (10, 10),
 (11, 11)]

# Calculate g(r)

In [130]:
def g_of_r(atom_pos, dr, r_max):
    """
    atom_pos :: N x 3 array of atom positions
    dr :: bin thickness
    """
    
    n = atom_pos.shape[0]
    r_vals = np.arange(0, r_max, dr)
    num_bins = int(np.floor(r_max / dr))
    diff = np.zeros(n*(n-1))

    i, j = np.meshgrid(range(n), range(n))
    i = i.reshape(n**2)
    j = j.reshape(n**2)
    
    diff = np.linalg.norm(atom_pos[i,:] - atom_pos[j,:], axis=1)
    diff = diff[np.nonzero(diff)]
    
    g_vals = np.histogram(diff, bins=num_bins, density=True)
    
    return r_vals, g_vals

In [133]:
atoms = np.random.rand(10000,3)

In [135]:
np.histogram?

In [134]:
g_of_r(atoms, .1, 1)

(array([ 0. ,  0.1,  0.2,  0.3,  0.4,  0.5,  0.6,  0.7,  0.8,  0.9]),
 (array([ 1579002,  8606786, 17164070, 22779114, 23040816, 17232322,
          7490196,  1878770,   210814,     8110]),
  array([  9.19967447e-04,   1.66308332e-01,   3.31696698e-01,
           4.97085063e-01,   6.62473428e-01,   8.27861793e-01,
           9.93250158e-01,   1.15863852e+00,   1.32402689e+00,
           1.48941525e+00,   1.65480362e+00])))

array([[0, 1, 2, 3, 4, 5],
       [0, 1, 2, 3, 4, 5],
       [0, 1, 2, 3, 4, 5],
       [0, 1, 2, 3, 4, 5],
       [0, 1, 2, 3, 4, 5],
       [0, 1, 2, 3, 4, 5]])

In [102]:
g_of_r(atoms, .1, 1)

KeyboardInterrupt: 

# Matt's Code to generate notebook

In [44]:
# get the notebook server base url
from notebook import notebookapp
server = list(notebookapp.list_running_servers())[0]

# get the specs for all installed kernels
import requests
import json
header = {'Authorization': 'token {}'.format(server["token"])}
response = requests.get('{}api/kernelspecs'.format(server["url"]),
                       headers=header)
kernelspecs = response.json()["kernelspecs"]

# create minimum info needed to specify kernel when saving a notebook as of 7/18/17
specs = {}
for k in kernelspecs:
    specs[k] = {
        "display_name": kernelspecs[k]["spec"]["display_name"],
        "language": kernelspecs[k]["spec"]["language"],
        "name": k
    }
print("kernels: {}".format(list(specs.keys())))

# Use the Notebook Server Contents REST API to save a notebook where the kernel is specified
# NOTE: You must use PUT and specify a filename.  Using POST yields the default kernel.
# Using PUT requires that you provide the name of the file to save.
# The kernel will be started in the notebook directory configured for the Notebook Server.
response = requests.put('{}api/contents/test_python_kernel.ipynb'.format(server["url"]), 
                         headers=header,
                         data=json.dumps(
                             {"content": {
                                 "cells": [],
                                 "metadata": {"kernelspec": specs["python3"]},
                                 "nbformat": 4,
                                 "nbformat_minor": 2
                             },
                              "type": "notebook"}))
print(response.json())

JSONDecodeError: Expecting value: line 1 column 1 (char 0)